In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer

2022-11-29 15:19:19.284048: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 15:19:19.618233: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-29 15:19:20.310880: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64::/usr/local/cuda/lib64:
2022-11-29 15:19:20.310989: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror:

In [2]:
df = pd.read_csv("/home/k0c/VIT/IMDB Dataset.csv")
df['sentiment'] = df['sentiment'].apply(lambda x: 1 if x=='positive' else 0)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [3]:
MODEL_PATH = 'bert-base-cased'

In [4]:
tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)

In [5]:
def encode(data,tokenizer):
    input_ids = []
    attention_masks = []
    token_type_ids = []
    for text in data:
        tokenized_text = tokenizer.encode_plus(text,max_length=50,add_special_tokens=True,pad_to_max_length=True,return_attention_mask=True)
        input_ids.append(tokenized_text['input_ids'])
        attention_masks.append(tokenized_text['attention_mask'])
        token_type_ids.append(tokenized_text['token_type_ids'])

    return input_ids, attention_masks, token_type_ids

In [6]:
from transformers import TFBertForSequenceClassification

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train,X_test,y_train,y_test = train_test_split(df['review'],df['sentiment'],test_size=0.2,random_state=42)

In [9]:
train_input_ids,train_attention_masks,train_token_type_ids = encode(X_train,tokenizer)
test_input_ids,test_attention_masks,test_token_type_ids = encode(X_test,tokenizer)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/k0c/.conda/envs/0chae/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2308: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [10]:
def map_example_to_dict(input_ids,attention_masks,token_type_ids,label):
    return {'input_ids':input_ids,'token_type_ids':token_type_ids,'attention_mask':attention_masks},label

In [11]:
def data_encode(input_ids_list,attention_mask_list,token_type_ids_list,label_list):
    return tf.data.Dataset.from_tensor_slices((input_ids_list,attention_mask_list,token_type_ids_list,label_list)).map(map_example_to_dict)

In [12]:
BATCH_SIZE = 32
train_data_encoded = data_encode(train_input_ids,train_attention_masks,train_token_type_ids,y_train).shuffle(10000).batch(BATCH_SIZE)
test_data_encoded = data_encode(test_input_ids,test_attention_masks,test_token_type_ids,y_test).shuffle(10000).batch(BATCH_SIZE)

2022-11-29 15:22:12.018182: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-29 15:22:15.339255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9638 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:19:00.0, compute capability: 7.5
2022-11-29 15:22:15.340084: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9638 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:1a:00.0, compute capability: 7.5
2022-11-29 15:22:15.340536: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/repli

In [13]:
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
w = model.get_weights()

In [15]:
from tqdm import tqdm
new_weight_list = []
for i in tqdm(w):
    s = i.shape
    new_weight = np.random.normal(0,1,s)
    new_weight_list.append(new_weight)

100%|██████████| 201/201 [00:04<00:00, 46.96it/s]


In [16]:
model.set_weights(new_weight_list)

In [17]:
model.get_weights()

[array([[-0.9204606 , -0.85722584, -2.2046177 , ...,  1.3179451 ,
         -0.17815214, -0.90786606],
        [-0.8785695 ,  0.3201665 ,  0.53019136, ..., -1.4612657 ,
         -0.9773435 ,  0.4933055 ],
        [ 0.33508575,  1.898007  ,  0.20772175, ...,  0.72983414,
          0.5146347 ,  3.1390994 ],
        ...,
        [-0.45967904, -0.8086509 , -0.7133568 , ..., -0.17403245,
         -0.834092  ,  0.01801772],
        [-0.0111293 , -1.7245212 ,  0.19877054, ..., -0.578258  ,
          0.29773223,  0.40994674],
        [-3.3430471 , -0.6301634 , -0.52279675, ...,  1.5772973 ,
         -0.30833107,  0.52134585]], dtype=float32),
 array([[-1.5698047 ,  0.7865751 ,  1.0042473 , ...,  0.57937443,
         -0.8096347 ,  0.8987694 ],
        [-1.2455074 , -1.3360887 , -1.0497559 , ..., -0.7130603 ,
          0.96702397, -0.22131906]], dtype=float32),
 array([[ 1.1875027 ,  0.3591348 , -1.136511  , ...,  0.9105677 ,
         -0.07098379, -1.2301824 ],
        [-0.35096338,  2.2131474 , 

In [18]:
optimizer = tf.keras.optimizers.Adam(2e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [19]:
NUM_EPOCHS = 1000
history = model.fit(train_data_encoded, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=test_data_encoded)

Epoch 1/1000
1250/1250 [==============================] - 203s 151ms/step - loss: 9.0098 - accuracy: 0.5027 - val_loss: 2.1271 - val_accuracy: 0.4961
Epoch 2/1000
1250/1250 [==============================] - 187s 149ms/step - loss: 7.9796 - accuracy: 0.4982 - val_loss: 5.7766 - val_accuracy: 0.5039
Epoch 3/1000
1250/1250 [==============================] - 187s 149ms/step - loss: 7.4219 - accuracy: 0.5024 - val_loss: 1.6542 - val_accuracy: 0.4961
Epoch 4/1000
 541/1250 [===========>..................] - ETA: 1:36 - loss: 7.1578 - accuracy: 0.4956

In [ ]:
model.evaluate(test_data_encoded)

313/313 [==============================] - 17s 54ms/step - loss: 1.3457 - accuracy: 0.5039


[1.3457319736480713, 0.5038999915122986]

In [79]:
NUM_EPOCHS = 100
history = model.fit(train_data_encoded, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, validation_data=test_data_encoded)

Epoch 1/100
1250/1250 [==============================] - 185s 148ms/step - loss: 6.5110 - accuracy: 0.4994 - val_loss: 1.3291 - val_accuracy: 0.5039
Epoch 2/100
1250/1250 [==============================] - 185s 148ms/step - loss: 6.3465 - accuracy: 0.4987 - val_loss: 2.1886 - val_accuracy: 0.5039
Epoch 3/100
1250/1250 [==============================] - 185s 148ms/step - loss: 6.2099 - accuracy: 0.4969 - val_loss: 1.6534 - val_accuracy: 0.5039
Epoch 4/100
1250/1250 [==============================] - 188s 150ms/step - loss: 6.0213 - accuracy: 0.5014 - val_loss: 0.7653 - val_accuracy: 0.5039
Epoch 5/100
1250/1250 [==============================] - 186s 148ms/step - loss: 5.9429 - accuracy: 0.5012 - val_loss: 0.7981 - val_accuracy: 0.5039
Epoch 6/100
1250/1250 [==============================] - 186s 149ms/step - loss: 5.8012 - accuracy: 0.5035 - val_loss: 0.8307 - val_accuracy: 0.4961
Epoch 7/100
1250/1250 [==============================] - 186s 149ms/step - loss: 5.8033 - accuracy: 0.4986

In [80]:
model.evaluate(test_data_encoded)

313/313 [==============================] - 17s 54ms/step - loss: 0.6962 - accuracy: 0.5039


[0.6961926221847534, 0.5038999915122986]